## Link Prediction

Link prediction is the problem of predicting the existence of a link between two nodes in a network.

<center><img src="images/link_prediction_problem.png" width="300"></center>

It has many applications such as: 
- **friend recommendation in social networks:** The goal is to predict which pairs of users, who are not currently friends, are most likely to become friends based on the existing structure of the network and possibly other user-specific data.
- **co-authorship prediction in citation networks:** Link prediction in co-authorship prediction within citation networks refers to the process of forecasting potential future collaborations between authors based on the existing structure and patterns of the network. In this context, a citation network is a graph where nodes represent authors (or papers), and edges represent co-authorship or citation relationships between them.
- **movie recommendation in Netflix:** Link prediction in the context of movie recommendation on platforms like Netflix involves predicting the likelihood of a user forming a "link" (i.e., an interaction such as watching, rating, or liking) with a movie they have not yet interacted with. The goal is to recommend movies that the user is most likely to enjoy based on their past behavior and the behavior of other users with similar preferences.
- **protein interaction prediction in biological networks:** Protein interaction prediction in biological networks, using link prediction, involves forecasting potential interactions between proteins within a biological network. This is crucial for understanding the molecular mechanisms underlying various biological processes and for identifying new targets for drug discovery.
- **drug response prediction:** Link prediction in drug response prediction involves forecasting how different drugs will interact with various targets (such as proteins, genes, or cells) and how these interactions will result in a therapeutic response or adverse effect. The primary goal is to predict the effectiveness or toxicity of a drug on a specific biological target, which can help in personalized medicine, drug discovery, and understanding disease mechanisms.

### Traditional Link Prediction Methods

These methods can be categorized into three classes: 
- heuristic methods 
- latent-feature methods
- content-based methods

The link prediction problem has been studied extensively, leading to the development of numerous techniques. 

___

We will first explore popular heuristics that utilize local and global neighborhood information. Can you think of a simple rule of thumb to predict whether two nodes should be connected?

        
- Common neighbors (CN): It is based on intuition that "the more neighbors you have in common, the more likely you are to be connected". This heuristic simply counts the number of neighbors two nodes ($x$ and $y$) have in common:

\begin{equation*}
f_{CN}(x,y) = |\mathcal{N}(x) \cap \mathcal{N}(y)|
\end{equation*}

- Jaccard coefficient: measures the proportion of shared neighbors between two nodes. It builds on the idea of common neighbors but normalizes the count by the total number of neighbors. This method favors nodes with fewer neighbors over those with a high degree.

\begin{equation*}
f_{Jaccard}(x,y) = \frac{|\mathcal{N}(x) \cap \mathcal{N}(y)|}{|\mathcal{N}(x) \cup \mathcal{N}(y)|}
\end{equation*}

- Preferential attachment (PA): measures link likelihood using the product of node degrees. PA assumes that node x is more likely to connect to node y if y has a high degree. For instance, in citation networks, a new paper is more likely to cite papers that already have many citations.

- Adamic-Adar (AA): evaluates the strength of the link likelihood by considering the weight of common neighbors between two nodes. Specifically, each common neighbor $z$ contributes to the score, but its influence is reduced based on its degree, with the contribution being inversely proportional to the logarithm of its degree $ \frac{1}{\log|G(z)|} $. This approach down-weights high-degree common neighbors, under the assumption that common neighbors with many connections are less indicative of a potential link between the nodes in question.

Note: The Adamic-Adar (AA) heuristic is considered a second-order heuristic because it evaluates the likelihood of a link between two nodes by considering not just the direct properties of the nodes themselves but also the properties of their shared neighbors (i.e., second-order neighbors).

Three heuristics of CN, PA, and AA are illustrated in the following figure:
<center><img src="images/link_prediction_heuristic.png" width="600"></center>
<center><small>image from https://graph-neural-networks.github.io/static/file/chapter10.pdf</small></center>

---

"One of the most popular latent feature methods is **matrix factorization**. This approach factorizes the observed adjacency matrix $A$ of the network into the product of a low-rank latent embedding matrix $Z \in \mathbb{R}^{n \times k}$ and its transpose (where $n$ is the number of nodes and $k$ is the dimension of hidden embeddings). Essentially, it approximates the edge between nodes $i$ and $j$ using their $k$-dimensional latent embeddings, $z_i$ and $z_j$: $\hat{A}_{i,j} = z^T_i z_j$.
<center><img src="images/matrix factorization concept.png" width=500></center>

Intuitively, if nodes $i$ and $j$ are similar, the dot product $z_i^T z_j$ should be high, indicating a likely link between them. This dot product can be used to approximate each element (link) in the adjacency matrix.

The method then minimizes the mean squared error between the reconstructed adjacency matrix and the true adjacency matrix over the observed edges to learn these latent embeddings: $L= \sum_{(i,j) \in E} (A_{i,j}-\hat{A}_{i,j})^2$.

---

However, both heuristic methods and latent-feature methods rely solely on the network’s topology and do not incorporate node features when creating embeddings. Content-based methods use specific features associated with nodes to predict links, which is common in recommender systems. For example, in citation networks, the words used in papers can serve as features. In social networks, a user’s profile details, like their age or interests, are used as features. However, content-based methods often perform worse than heuristic and latent-feature methods because they don’t use the connections between nodes in the graph.


## Link prediction with GNN

In contrast to the traditional methods that either focus on graph structure or node features, GNN methods integrate both graph structure features and content features by learning them simultaneously in a unified framework, taking advantage of the powerful graph representation learning capabilities of GNNs.

There are two main paradigms for link prediction using GNNs: node-based and subgraph-based. **Node-based** methods aggregate the node representations learned by a GNN to form the link representation. In contrast, **subgraph-based** methods extract a local subgraph surrounding each potential link and use the representation of this subgraph, learned by a GNN, as the link representation.

### Node based GNN


The foundational node-based method for link prediction is the Graph AutoEncoder (GAE), introduced by Kipf and Welling in 2016. GAE is designed to learn low-dimensional node embeddings from graph data, which can then be used to predict the existence of links between nodes. Given a graph's adjacency matrix $A$ and node feature matrix $X$, GAE employs a Graph Convolutional Network (GCN) to compute a node representation $z_i$ for each node $i$:

\begin{equation*}
\hat{A}_{ij} = \sigma(z_i^T z_j), where \ z_i = Z[i,:], Z = GCN(X,A)
\end{equation*}

The prediction of a link between nodes $i$ and $j$ is based on the similarity score $s(z_i^T z_j)$, where the dot product between their embeddings $z_i$ and $z_j$ is used to estimate the likelihood of an edge between them. The specific prediction for the link is given by:

\begin{equation*}
\hat{A}_{ij} = \sigma(z_i^T z_j)
\end{equation*}

where:
- $Z$ is the matrix of node representations output by the GCN, with the $i$-th row $z_i = Z[i,:]$ representing the embedding for node $i$.
- $\hat{A}_{ij}$ is the predicted probability of a link between nodes $i$ and $j$.
- $\sigma$ is the sigmoid function, which ensures that the predicted probabilities lie between 0 and 1.

If no node feature matrix $X$ is provided, GAE can use a one-hot encoded identity matrix $I$ as input instead, treating each node's identity as its feature.

The GAE model is trained to minimize the cross-entropy loss between the reconstructed adjacency matrix $\hat{A}$ and the true adjacency matrix $A$. The loss function is defined as:

\begin{equation*}
L = \sum_{i \in V, j \in V} \left[ -A_{ij} \log \hat{A}_{ij} - (1 - A_{ij}) \log (1 - \hat{A}_{ij}) \right]
\end{equation*}

This loss function encourages the model to accurately reconstruct the adjacency matrix, effectively learning node embeddings that capture the underlying graph structure and are useful for predicting missing links.

<center><img src="images/GAE.png" width=700></center>

In practice, the loss for positive edges (where $A_{i,j} = 1$) is increased by a factor of $k$, which is the ratio of negative edges (where $A_{i,j} = 0$) to positive edges. This adjustment helps balance the impact of positive and negative edges on the overall loss. Without this, the loss could be overwhelmed by negative edges because real-world networks are often sparse, meaning there are many more negative edges than positive ones.

##### Does GAE Just Relearn the Actual $A$?
It might seem like GAE (Graph AutoEncoder) is just trying to reconstruct the actual adjacency matrix $A$, but its real power lies in how it learns to generalize from the data.
While GAE does try to reconstruct the adjacency matrix $A$, it's not just memorizing it. Instead, it's learning patterns and structures within the graph. By compressing the graph structure into a lower-dimensional space, the model can generalize to predict links that are not directly observed in the training data.

Hence, While GAE does aim to reconstruct the adjacency matrix $A$ during training, it's doing so by learning a compressed, latent representation of the graph. The model tries to capture the underlying structure and patterns, not just memorize the existing connections. This learned representation can then be used to make predictions about new, unseen links that weren’t present in the original matrix. In essence, GAE generalizes from the observed data to infer the likelihood of connections between any pair of nodes, whether or not those connections were in the original graph.

### Subgraph-based GNN